<a href="https://colab.research.google.com/github/blockchainrelativity/Colab_Experiments/blob/main/LangChain_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

os.environ["OPENAI_API_KEY"] = ''
os.environ["SERPAPI_API_KEY"] = ''

# Initialize the chat model and other components
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="Useful for answering questions about current events or the current state of the world. The input should be a single search term."
    ),
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent='chat-conversational-react-description', verbose=True, memory=memory)

# Prompt templates for collecting user skills and years of experience
skills_prompt_template = HumanMessagePromptTemplate.from_template("Please provide your skills:")
experience_prompt_template = HumanMessagePromptTemplate.from_template("Please enter your years of experience:")

# Create a chat prompt template with system message and user inputs
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Hello! I am here to assist you with your skills and capabilities."),
    skills_prompt_template,
    experience_prompt_template
])

# Collect user skills and years of experience
response = agent_chain.run(input=chat_prompt.format_prompt().to_messages())

# Extract user inputs from the response
user_skills = response['messages'][-2]['content']
years_of_experience = response['messages'][-1]['content']

# Provide feedback based on user input
feedback = ""
if len(user_skills.split(',')) >= 3:
    feedback += "Great! You have a strong set of skills. "
else:
    feedback += "You have a decent set of skills. "

if int(years_of_experience) >= 5:
    feedback += "Your years of experience demonstrate a solid foundation. "
else:
    feedback += "With more experience, you can further strengthen your capabilities. "

# Assist the user with recommendations and advice
assistance_prompt_template = HumanMessagePromptTemplate.from_template(f"{feedback} How can I assist you further?")
chat_prompt_assistance = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Sure! Here are some recommendations and advice:"),
    assistance_prompt_template
])

# Ask for further assistance and provide recommendations
response_assistance = agent_chain.run(input=chat_prompt_assistance.format_prompt().to_messages())

# Extract user's response to assistance prompt
user_assistance_response = response_assistance['messages'][-1]['content']

# Process user's response and provide appropriate assistance
# ... (Add your own logic here based on the user's response)

# Print the agent's response
print(response_assistance['messages'][-1]['content'])
